# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    
    #Clean up cities
    def camel_case_split(str):
        return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)
    
    keep1 = ['Metropolitan area', 'NHL', 'Population (2016 est.)[8]'] 
    cities2 = cities[keep1]
    cities2 = cities2.replace({'NHL': '\[.*\]'}, {'NHL' : ''}, regex = True)\
                .replace({'NHL': '—'}, {'NHL' : np.nan}, regex = True)\
                .dropna()

    cities2['Split'] = cities2['NHL'].apply(lambda x: camel_case_split(x))
    cities2 = cities2.Split.apply(pd.Series) \
                .merge(cities2, right_index = True, left_index = True) \
                .melt(id_vars = ['Metropolitan area', 'Split', 'NHL', 'Population (2016 est.)[8]'], value_name = "Team")
    cities2 = cities2.dropna().drop(['Split', 'NHL', 'variable'], axis=1).set_index('Team')
    
    
    #Clean up NHL
    keep2 = ['team','W','L','year']
    nhl_df2 = nhl_df[(~nhl_df.team.str.contains('Division')) & (nhl_df.year == 2018)][keep2]
    nhl_df2 = nhl_df2.replace({'team': '\*'}, {'team' : ''}, regex = True)
    nhl_df2['teamx'] = nhl_df2['team'].str.split().str[-1]
    nhl_df2 = nhl_df2.set_index('teamx')
    
    #Merge on truncated team name
    merged = nhl_df2.merge(cities2, how='left', left_index = True, right_index=True)
    merged['Win Loss Ratio']= merged['W'].astype('int')/(merged['W'].astype('int')+merged['L'].astype('int'))
    merged['Population (2016 est.)[8]'] = merged['Population (2016 est.)[8]'].astype('int')
    
    wlratio = merged.groupby('Metropolitan area')['Win Loss Ratio'].agg('mean')
    pop = merged.groupby('Metropolitan area')['Population (2016 est.)[8]'].agg('mean')
    
    #raise NotImplementedError()
    
    population_by_region = pop # pass in metropolitan area population from cities
    win_loss_by_region = wlratio # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    
    # Clean up Cities
    def camel_case_split(str):
        return re.findall(r'[A-Z\d](?:[a-z\d]+|[A-Z]*(?=[A-Z]|$))', str)
    
    keep1 = ['Metropolitan area', 'NBA', 'Population (2016 est.)[8]'] 
    cities2 = cities[keep1]
    cities2 = cities2.replace({'NBA': '\[.*\]'}, {'NBA' : ''}, regex = True)\
                .replace({'NBA': '—'}, {'NHL' : np.nan}, regex = True)\
                .dropna()

    cities2['Split'] = cities2['NBA'].apply(lambda x: camel_case_split(x))
    cities2 = cities2.Split.apply(pd.Series) \
                .merge(cities2, right_index = True, left_index = True) \
                .melt(id_vars = ['Metropolitan area', 'Split', 'NBA', 'Population (2016 est.)[8]'], value_name = "Team")
    cities2 = cities2.dropna().drop(['Split', 'NBA', 'variable'], axis=1).set_index('Team')
    
    #Clean up NBA
    keep2 = ['team','W','L','year']
    nba_df2 = nba_df[(~nba_df.team.str.contains('Division')) & (nba_df.year == 2018)][keep2]
    nba_df2 = nba_df2.replace(to_replace = r'\*?\s\(.*\)', value = '', regex = True)
    nba_df2['teamx'] = nba_df2['team'].str.split().str[-1]
    nba_df2 = nba_df2.set_index('teamx')
    
    #Merge on truncated team name
    merged = nba_df2.merge(cities2, how='left', left_index = True, right_index=True)
    merged['Win Loss Ratio']= merged['W'].astype('int')/(merged['W'].astype('int')+merged['L'].astype('int'))
    merged['Population (2016 est.)[8]'] = merged['Population (2016 est.)[8]'].astype('int')
    
    wlratio = merged.groupby('Metropolitan area')['Win Loss Ratio'].agg('mean')
    pop = merged.groupby('Metropolitan area')['Population (2016 est.)[8]'].agg('mean')
    
    population_by_region = pop # pass in metropolitan area population from cities
    win_loss_by_region = wlratio # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]


    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # Clean up Cities
    def camel_case_split(str):
        return re.findall(r'[A-Z\d](?:[a-z\d]+|[A-Z]*(?=[A-Z]|$))', str)
    
    keep1 = ['Metropolitan area', 'MLB', 'Population (2016 est.)[8]'] 
    cities2 = cities[keep1]
    cities2 = cities2.replace({'MLB': '\[.*\]'}, {'MLB' : ''}, regex = True)\
                .replace({'MLB': '—'}, {'MLB' : np.nan}, regex = True)\
                .dropna()

    cities2['Split'] = cities2['MLB'].apply(lambda x: camel_case_split(x))
    cities2 = cities2.Split.apply(pd.Series) \
                .merge(cities2, right_index = True, left_index = True) \
                .melt(id_vars = ['Metropolitan area', 'Split', 'MLB', 'Population (2016 est.)[8]'], value_name = "Team")
    cities2 = cities2.dropna().drop(['Split', 'MLB', 'variable'], axis=1).set_index('Team')
    cities2 = cities2.rename(index = {'Red' : 'Red Sox', 'White' : 'White Sox'})
    
    #Clean up MLB
    keep2 = ['team','W','L','year']
    mlb_df2 = mlb_df[(~mlb_df.team.str.contains('Division')) & (mlb_df.year == 2018)][keep2]
    mlb_df2['teamx'] = mlb_df2['team'].str.split().str[-1]
    mlb_df2.loc[0,'teamx'] = 'Red Sox'
    mlb_df2.loc[8, 'teamx'] = 'White Sox'
    mlb_df2 = mlb_df2.set_index('teamx')
    
    
    #Merge on truncated team name
    merged = mlb_df2.merge(cities2, how='left', left_index = True, right_index=True)
    merged['Win Loss Ratio']= merged['W'].astype('int')/(merged['W'].astype('int')+merged['L'].astype('int'))
    merged['Population (2016 est.)[8]'] = merged['Population (2016 est.)[8]'].astype('int')
    
    wlratio = merged.groupby('Metropolitan area')['Win Loss Ratio'].agg('mean')
    pop = merged.groupby('Metropolitan area')['Population (2016 est.)[8]'].agg('mean')
    
    population_by_region = pop # pass in metropolitan area population from cities
    win_loss_by_region = wlratio # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # Clean up Cities
    def camel_case_split(str):
        return re.findall(r'[A-Z\d](?:[a-z\d]+|[A-Z]*(?=[A-Z]|$))', str)
    
    keep1 = ['Metropolitan area', 'NFL', 'Population (2016 est.)[8]'] 
    cities2 = cities[keep1]
    cities2 = cities2.replace({'NFL': '\[.*\]'}, {'NFL' : ''}, regex = True)\
                .replace({'NFL': '—'}, {'NFL' : np.nan}, regex = True)\
                .dropna()

    cities2['Split'] = cities2['NFL'].apply(lambda x: camel_case_split(x))
    cities2 = cities2.Split.apply(pd.Series) \
                .merge(cities2, right_index = True, left_index = True) \
                .melt(id_vars = ['Metropolitan area', 'Split', 'NFL', 'Population (2016 est.)[8]'], value_name = "Team")
    cities2 = cities2.dropna().drop(['Split', 'NFL', 'variable'], axis=1).set_index('Team')
    
    
    #Clean up NFL
    keep2 = ['team','W','L','year']
    nfl_df2 = nfl_df[(~nfl_df.team.str.contains('AFC|NFC')) & (nfl_df.year == 2018)][keep2]
    nfl_df2 = nfl_df2.replace(to_replace = r'\*|\+', value = '', regex = True)
    nfl_df2['teamx'] = nfl_df2['team'].str.split().str[-1]
    nfl_df2 = nfl_df2.set_index('teamx')
    
    #Merge on truncated team name
    merged = nfl_df2.merge(cities2, how='left', left_index = True, right_index=True)
    merged['Win Loss Ratio']= merged['W'].astype('int')/(merged['W'].astype('int')+merged['L'].astype('int'))
    merged['Population (2016 est.)[8]'] = merged['Population (2016 est.)[8]'].astype('int')
    
    wlratio = merged.groupby('Metropolitan area')['Win Loss Ratio'].agg('mean')
    pop = merged.groupby('Metropolitan area')['Population (2016 est.)[8]'].agg('mean')
    
    population_by_region = pop # pass in metropolitan area population from cities
    win_loss_by_region = wlratio # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [80]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [82]:
def clean_cities():
    
    def camel_case_split(str):
        return re.findall(r'[A-Z\d](?:[a-z\d]+|[A-Z]*(?=[A-Z]|$))', str)
    
    keep1 = ['Metropolitan area', 'NFL', 'MLB', 'NBA', 'NHL', 'Population (2016 est.)[8]'] 
    cities2 = cities[keep1]
    cities2 = cities2.replace(regex = '\[.*\]', value='')\
                .replace(regex = '—', value=np.nan)\
    
    sports = ['MLB', 'NBA', 'NHL', 'NFL']
    labels = []
    for sport in sports:
        label = 'Split' + sport
        labels.append(label)
        cities2[label] = cities2[sport].dropna().apply(lambda x: camel_case_split(x))
        cities2 = cities2.drop(columns = [sport])
    
    for label in labels:
        cities2 = cities2[label].apply(pd.Series) \
                .merge(cities2, right_index = True, left_index = True) \
                .melt(id_vars = ['Metropolitan area', 'Population (2016 est.)[8]', 'SplitMLB', 'SplitNBA', 'SplitNFL', 'SplitNHL'], value_name = "Team")
        
        if label == 'SplitMLB':
            cities2 = cities2.replace({'White':'White Sox', 'Red':'Red Sox', 'Rangers':'Rangersx', 'Cardinals': 'Cardinalsx', 'Giants':'Giantsx'})
            
        if label == 'SplitNHL':
            cities2 = cities2.replace({'Jets':'Jetsx', 'Panthers':'Panthersx'})
            
        if label == 'SplitNBA':
            cities2 = cities2.replace({'Kings':'Kingsx'})
    
    cities2 = cities2.drop(columns=['SplitMLB','SplitNBA','SplitNFL','SplitNHL','variable']).drop_duplicates('Team').dropna()\
                    .set_index('Team')
    
    return cities2

In [84]:
def clean_sport(sport_df):

    keep2 = ['team','W','L','year']
    sport_df = sport_df[(~sport_df.team.str.contains('Division|AFC|NFC')) & (sport_df.year == 2018)][keep2]
    sport_df = sport_df.replace({'team': '\*|\*?\s\(.*\)|\+'}, {'team' : ''}, regex = True)
    sport_df['teamx'] = sport_df['team'].str.split().str[-1]
    
    if sport_df.teamx.str.contains('Sox').any():
        sport_df.loc[0,'teamx'] = 'Red Sox'
        sport_df.loc[8, 'teamx'] = 'White Sox'
        sport_df = sport_df.replace({'Rangers':'Rangersx', 'Cardinals': 'Cardinalsx', 'Giants':'Giantsx'})
        
    if sport_df.teamx.str.contains('Leafs').any():
        sport_df = sport_df.replace({'Jets':'Jetsx', 'Panthers':'Panthersx'})
        
    if sport_df.teamx.str.contains('Knicks').any():
        sport_df = sport_df.replace({'Kings':'Kingsx'})
    
    sport_df = sport_df.set_index('teamx')
    
    sport_df['Win Loss Ratio']= sport_df['W'].astype('int')/(sport_df['W'].astype('int')+sport_df['L'].astype('int'))
    sport_df = sport_df.drop(columns = ['W', 'L', 'year'])
    
    return sport_df

In [85]:
def merge_city_sport(sport_df):
    
    #Merge on truncated team name
    cities2 = clean_cities()
    sport_df = clean_sport(sport_df)
    
    merged = sport_df.merge(cities2, how='left', left_index = True, right_index=True)
    merged['Population (2016 est.)[8]'] = merged['Population (2016 est.)[8]'].astype('int')
    
    merged = merged.groupby('Metropolitan area')['Win Loss Ratio', 'Population (2016 est.)[8]'].mean()

    return merged
    

In [86]:
df_dict = {'NFL':nfl_df, 'NHL': nhl_df, 'MLB':mlb_df, 'NBA':nba_df}
nameslst = []
for sport in ['NHL', 'NBA', 'MLB', 'NFL']:
    x = clean_sport(df_dict[sport])
    y = (x.index.unique().tolist())
    for name in y:
        nameslst.append(name)

uniqueteam = np.unique(nameslst)


In [90]:
merge_city_sport(nba_df)

,Win Loss Ratio,Population (2016 est.)[8]
Metropolitan area,,
Atlanta,0.292683,5789700
Boston,0.670732,4794447
Charlotte,0.439024,2474314
Chicago,0.329268,9512999
Cleveland,0.609756,2055612
Dallas–Fort Worth,0.292683,7233323
Denver,0.560976,2853077
Detroit,0.475610,4297617
Houston,0.792683,6772470


In [93]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]



def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    df_dict = {'NFL':nfl_df, 'NHL': nhl_df, 'MLB':mlb_df, 'NBA':nba_df}
    
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for index in p_values.index:
        for column in p_values.columns:
            
            #Merge two sports on Metropolitan Area
            sport1 = merge_city_sport(df_dict[index])
            sport2 = merge_city_sport(df_dict[column])
            
            merged = sport1.merge(sport2, how = 'inner', on = 'Metropolitan area')
            
            p_values.loc[index,column] = (stats.ttest_rel(merged['Win Loss Ratio_x'], merged['Win Loss Ratio_y']))[1]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [94]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
